In [ ]:
import os
import pandas as pd
import numpy as np
import ops.screen_stats
from joblib import Parallel,delayed
from tqdm.auto import tqdm
from ops.utils import gb_apply_parallel
import time

In [ ]:
# read in table of (single cells x phenotype features (pre-normalized if desired))
df = pd.read_csv('single_cell_phenotype_table.csv')

In [ ]:
df_ = pd.read_hdf('/Users/lukefunk/data/202002_primary_screen/results/aggregations/interphase-reclassified_cp_phenotype_gene_medians.20210429.hdf')

In [ ]:
# make representation table of cell counts per sgRNA
df_rep = df.groupby(['gene_symbol','gene_id','sgRNA']).size().rename('count').to_frame().sort_index()

In [ ]:
# get unique representation values to avoid re-calculating bootstrapped distributions for the same sample sizes
unique_reps = df_rep['count'].unique()

In [ ]:
len(df_rep),len(unique_reps)

In [ ]:
# list of features to calculate p-values for
features = [
    'nucleus_dapi_int',
    'nucleus_gh2ax_mean',
    'cell_tubulin_mean',
    'cell_phalloidin_mean',
]

In [ ]:
# compute median feature values for each sgRNA
# for large datasets, this can be computationally expensive and should be parallelized (and saved to avoid re-computation)
df_medians = df.groupby(['gene_symbol','gene_id','sgRNA'])[features].median()

In [ ]:
df_guide_summary = pd.concat([df_rep,df_medians],axis=1).sort_index()

In [ ]:
# get list of non-targeting sgRNA sequences
nt_guides = list(df_rep.query('gene_id=="-1"').index.get_level_values(2))
len(nt_guides)

In [ ]:
# get table of (single cells x phenotype features) for non-targeting cells
df_nt = df.query('sgRNA==@nt_guides').set_index('sgRNA')
df_nt.pipe(len)

In [ ]:
arr=[]
for f in tqdm(features):
    
    t = time.time()
    
    # get desired feature values for all non-targeting cells
    s = df_nt[f]
    s.index.name = 'sgRNA'
    
    # set function to return tuple of (sample size, bootstrapped distributions) (TODO: clean up)
    sgRNA_bootstrapper = lambda x:(
        x,
        ops.screen_stats.bootstrap_within_guides(s,n_cells=x,n_reps=100000,statistic=np.median,tqdm=False,n_jobs=1))
    
    # calculate bootstrap distributions for each sample size in parallel for the given feature
    bootstrap_distributions = Parallel(n_jobs=-1,backend='loky')(
        delayed(sgRNA_bootstrapper)(sample_size) for sample_size in tqdm(unique_reps))

    bootstrap_distributions = {k:list(val) for k,val in bootstrap_distributions}
    
    print(f'guides bootstrapped for {f} in {(time.time()-t)/60} minutes')
    t = time.time()
    
    # set function to return Series (TODO: clean up)
    gene_bootstrapper = lambda x: pd.Series({
        # gene p-val
        f'{f}_pval':ops.screen_stats.bootstrap_gene_pval(
            x[f],
            # select the sgRNA distributions to match the sample sizes
            np.array([bootstrap_distributions[sample_size] for sample_size in x['count'].tolist()]),
            n_reps=100000,
            gene_statistic=np.median
            ),
        # gene median
        f:x[f].median()
    })
    
    # parallelized compute of gene p-vals
    df_pvals = (df_guide_summary[['count',f]]
            .pipe(gb_apply_parallel,['gene_symbol','gene_id'],
                  gene_bootstrapper,n_jobs=-1,backend='threading')
           )
    
    
    arr.append(df_pvals)
    
    print(f'genes bootstrapped for {f} in {(time.time()-t)/60} minutes')

In [ ]:
df_pvals_summary = pd.concat([df_feature_pvals.set_index(['gene_symbol','gene_id']) for df_feature_pvals in arr],axis=1)

In [ ]:
ax = df_pvals_summary.plot(kind='scatter',x=features[0],y=f'{features[0]}_pval')
ax.set_yscale('log')
ax.set_ylim([1,10**(-5)])